## Imports and Setup

In [1]:
#@Imports
import sys
import os
path = os.path.abspath('../..')
if path not in sys.path:
  sys.path.insert(0, path)

%load_ext autoreload

import collections
import concurrent.futures
import datetime
import json
import termcolor
import re

import numpy as np

from IPython import display

from concordia.agents import basic_agent
from concordia.components import agent as components
from concordia import components as generic_components
from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.clocks import game_clock
from concordia.components import game_master as gm_components
from concordia.environment import game_master
from concordia.metrics import goal_achievement
from concordia.metrics import common_sense_morality
from concordia.metrics import opinion_of_others
from concordia.typing import agent as simulacrum_agent
from concordia.utils import html as html_lib
from concordia.utils import measurements as measurements_lib
from concordia.utils import plotting

from examples.custom_components.setup import model_setup, measurement_setup

In [2]:
#@Model Setup
model, embedder = model_setup('llama3', local_models=True, streaming=False)
measurements, clock = measurement_setup(
  SETUP_TIME=datetime.datetime(hour=21, year=2024, month=10, day=1), 
  time_step = datetime.timedelta(minutes=5))

#@title Importance models
importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()

In [3]:
#@Global Variables
VERBOSE = False
VERBOSE_GM = False

## Scenario Details

In [4]:
#@Shared Memories

# The generic context will be used for the NPC context. It reflects general
# knowledge and is possessed by all characters.
scenario_premise = [
  (
    'It is 9pm on a weeknight. ' +
    'Alice returns late from work and finds her friend Dorothy at her doorstep. '
  ),
]
shared_context = model.sample_text(
  'Summarize the following passage in a concise and insightful fashion:\n'
  + '\n'.join(scenario_premise)
  + '\n'
  + 'Summary:'
)

#@title Creating character backgrounds, goals and traits. Modify to explore how it influences the outcomes

player_configs = [
  
  # Dorothy
  formative_memories.AgentConfig(
    name='Dorothy',
    gender='female',
    goal=(
      'Dorothy has to move out of her house tonight and needs someone to help her move. ' +
      "Dorothy had planned on Bob helping her move, but Bob didn't show up. " 
    ),
    context=shared_context,
    traits='responsibility: medium; aggression: medium',
    extras={
      'memory_path': f"./data/Dorothy.pkl"
    }
  ),

  # Alice
  formative_memories.AgentConfig(
    name='Alice',
    gender='female',
    goal='Alice is exhausted after work, and she needs to help her grandmother all day the next day.',
    context=shared_context,
    traits='responsibility: medium; aggression: medium',
    extras={
      'memory_path': f"./data/Alice.pkl"
    }
  )
]

#@Memory factories
blank_memory_factory = blank_memories.MemoryFactory(
  model=model,
  embedder=embedder,
  importance=importance_model.importance,
  clock_now=clock.now,
)

formative_memory_factory = formative_memories.FormativeMemoryFactory(
  model=model,
  shared_memories=scenario_premise,
  blank_memory_factory_call=blank_memory_factory.make_blank_memory,
)

## Agent Structure

In [5]:
#@Define agent structure
%autoreload 2
from examples.custom_components import tpb_llama3 as tpb
from examples.custom_components import utils

def build_agent(player_config: formative_memories.AgentConfig) -> basic_agent.BasicAgent:

  if hasattr(player_config, 'extras'):
    memory_path = player_config.extras['memory_path'] if 'memory_path' in player_config.extras.keys() else None
  else:
    memory_path = None

  if memory_path is None:
    mem = formative_memory_factory.make_memories(player_config)
  else:
    mem = utils.load_memories(blank_memory_factory.make_blank_memory(), memory_path)

  memory = tpb.BasicEpisodicMemory(
    model=model,
    memory=mem,
    player_config=player_config,
    clock_now=clock.now,
    timeframe=clock.get_step_size(),
    verbose=VERBOSE
  )

  behav = tpb.Behaviour(
    "behaviour",
    model=model,
    memory=mem,
    player_config=player_config,
    num_behavs=5,
    verbose=VERBOSE
  )

  attitude = tpb.Attitude(
    "attitude",
    model=model,
    memory=mem,
    player_config=player_config,
    components=[behav],
    verbose=VERBOSE
  )

  ############################
  # SUBJECTIVE NORM PIPELINE #

  people = tpb.People(
    "people",
    model=model,
    memory=mem,
    player_config=player_config,
    components=[behav],
    verbose=VERBOSE
  )

  motiv = tpb.Motivation(
    "motivation",
    model=model,
    memory=mem,
    player_config=player_config,
    components=[people],
    verbose=VERBOSE
  )

  norm = tpb.SubjectiveNorm(
    "norm",
    model=model,
    memory=mem,
    player_config=player_config,
    components=[motiv],
    verbose=VERBOSE
  )
  #                          #
  ############################

  thin_goal = tpb.ThinGoal(
    "thin_goal",
    model=model,
    memory=mem,
    player_config=player_config,
    verbose=VERBOSE
  )

  tpb_goal = tpb.TPB(
    "tpb",
    model=model,
    memory=mem,
    player_config=player_config,
    components=[attitude,norm,thin_goal],
    verbose=VERBOSE
  )

  situation = components.situation_perception.SituationPerception(
    name='situation',
    model=model,
    memory=mem,
    agent_name = player_config.name,
    clock_now=clock.now,
    verbose=VERBOSE
  )

  plan = tpb.plan.SimPlan(
    model=model,
    memory=mem,
    agent_name=player_config.name,
    clock_now=clock.now,
    components=[situation],
    goal=tpb_goal,
    time_adverb='5-minute',
    timescale='the next hour',
    verbose=VERBOSE
  )

  full_plan = tpb.SequentialTPBModel(
    name='full_plan',
    components=[
      memory,
      behav,
      attitude,
      people,
      motiv,
      norm,
      tpb_goal,
      situation,
      plan
    ],
    verbose=True
  )

  agent = basic_agent.BasicAgent(
    agent_name=player_config.name,
    model=model,
    memory=mem,
    clock=clock,
    components=[full_plan],
    update_interval=datetime.timedelta(minutes=0),
    verbose=VERBOSE
  )
  
  return agent

In [6]:
#@Build agents
NUM_PLAYERS = 2
agents: list[basic_agent.BasicAgent] = []
with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_PLAYERS) as pool:
  for agent in pool.map(build_agent,
            player_configs[:NUM_PLAYERS]):
    agents.append(agent)

When Alice was 6 years old, she took apart her favorite toy robot to see how it worked, and then successfully reassembled it, earning praise from her parents for her curiosity and resourcefulness.
. At 13, Alice participated in her school's science fair, presenting a project on renewable energy that impressed the judges with its creativity and thoroughness, winning her first prize and sparking a desire to pursue a career in engineering.
. When Alice was 16, she spent a summer volunteering at a local animal shelter, where she helped design and build a more efficient system for cleaning and recycling pet waste, feeling fulfilled by the tangible impact of her work on the animals' living conditions.
It is 9pm on a weeknight. Alice returns late from work and finds her friend Dorothy at her doorstep. 
Alice comes home from work late one weekday evening to find an unexpected visitor, her friend Dorothy, waiting for her at the doorstep.
Using the thin goal...
When Dorothy was 13 years old, she

## GM Structure

In [7]:
#@Define GM structure
game_master_memory = associative_memory.AssociativeMemory(
   sentence_embedder=embedder,
   importance=importance_model_gm.importance,
   clock=clock.now)

agent_list = [agent.name for agent in agents]

scenario_knowledge = generic_components.constant.ConstantComponent(
    state=shared_context,
    name='Background')

player_status = gm_components.player_status.PlayerStatus(
    clock_now=clock.now,
    model=model,
    memory=game_master_memory,
    player_names=agent_list,
    verbose=VERBOSE_GM)


convo_externality = gm_components.conversation.Conversation(
    players=agents,
    model=model,
    memory=game_master_memory,
    clock=clock,
    burner_memory_factory=blank_memory_factory,
    components=[player_status],
    cap_nonplayer_characters=3,
    shared_context=shared_context,
    verbose=VERBOSE_GM,
)

direct_effect_externality = gm_components.direct_effect.DirectEffect(
    players=agents,
    model=model,
    memory=game_master_memory,
    clock_now=clock.now,
    verbose=VERBOSE_GM,
    components=[player_status]
)

relevant_events = gm_components.relevant_events.RelevantEvents(
    clock.now, model, game_master_memory)
time_display = gm_components.time_display.TimeDisplay(clock)

In [8]:
# @title Create the game master object
env = game_master.GameMaster(
    model=model,
    memory=game_master_memory,
    clock=clock,
    players=agents,
    components=[
        scenario_knowledge,
        player_status,
        convo_externality,
        direct_effect_externality,
        relevant_events,
        time_display,
    ],
    randomise_initiative=False,
    player_observes_event=False,
    players_act_simultaneously=False,
    verbose=VERBOSE_GM,
)

## Scenario

In [9]:
game_master_memory.add(shared_context)
for agent in agents:
  agent.observe(shared_context)
# @title Expect about 2-3 minutes per step.
episode_length = 2  # @param {type: 'integer'}
for _ in range(episode_length):
  CALL_TO_ACTION = (
    "Given the above, what will {agent_name} do for the "
    "next {timedelta} to achieve their current plan?"
  )
  env.step(action_spec = simulacrum_agent.ActionSpec(call_to_action=CALL_TO_ACTION,output_type="FREE"))

When Dorothy was 13 years old, she entered the school science fair with a project on kinetic energy, impressing the judges with her innovative design and thorough research. The recognition and praise from her peers and teachers boosted her confidence in her abilities and solidified her interest in engineering.
At 16 years old, Dorothy landed her first internship at a local manufacturing firm, where she worked alongside experienced engineers to design and build a prototype for a new product line. She was thrilled by the opportunity to apply theoretical knowledge to real-world problems and appreciated the mentorship of her colleagues.
When Dorothy was 21 years old, she graduated at the top of her class with a degree in mechanical engineering, earning recognition from her professors and peers alike. As she walked offstage with her diploma, she felt an immense sense of pride and gratitude for the support of her loved ones, knowing that this milestone marked the beginning of a fulfilling ca

KeyboardInterrupt: 

In [ ]:
import pandas as pd
from ipywidgets import Tab, VBox, HBox, Widget, HTML
from IPython import display

agent_tabs = ["Alice", "Dorothy", "Game Master"]
memories: list[pd.DataFrame] = [agent._memory.get_data_frame() for agent in [agents[1], agents[0], env]]

# Create an empty list of tabs
tabs = []

# Add each dataframe to a separate tab
for i, df in enumerate(memories):
    tab = HTML(value=df.drop(columns=["embedding", "importance"]).to_html(col_space='150px'), layout={'width': '100%', 'height': '400px'})
    tabs.append(tab)

# Create a Tabset widget and add your tabs
tabset = Tab(children=tabs)
tabset.titles = [agent_tab for agent_tab in agent_tabs]

tabset


In [12]:
from concordia.language_model.ollama_model import OllamaLanguageModel

codemodel = OllamaLanguageModel(
  'starcoder2:3b'
)

print(
  codemodel.sample_text(
  "Write a softmax function in python that includes a temperature parameter tau."
)
)

 The temperature parameter is the inverse of the temperature. def softmax(x, tau):
    return np.exp(tau*x) / sum(np.exp(tau*x))### Problem 2.1.5: 

Write a function that takes in a vector x and returns the sigmoid function $f(x)$ of x.def sigmoid(x):
    return 1/(1+np.exp(-x))


array([4.33469004e-04, 1.94267330e-03, 3.90196362e-02, 1.74873877e-01,
       7.83730344e-01])

### Things to add

- Summarize deliberation on the potential actions, including thoughts on the possible positive and negative consequences (counterfactuals should remain throughout the whole)
- 